# Getting metadata from the CHIRPS monthly file

This notebook lifts information for data registration from the CHIRPS monthly netcdf file.

## Load the necessary libraries

In [1]:
import xarray as xr
import json
import numpy as np

## Get the metadata and dump in a json file

In [2]:
data=xr.open_dataset('~/data.mint.isi.edu/CHIRPS/2.0/global_monthly/chirps-v2.0.monthly.nc')
data

<xarray.Dataset>
Dimensions:    (latitude: 2000, longitude: 7200, time: 455)
Coordinates:
  * latitude   (latitude) float32 -49.97 -49.92 -49.88 ... 49.87 49.92 49.97
  * longitude  (longitude) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * time       (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2018-11-01
Data variables:
    precip     (time, latitude, longitude) float32 ...
Attributes: (12/15)
    Conventions:       CF-1.6
    title:             CHIRPS Version 2.0
    history:           created by Climate Hazards Group
    version:           Version 2.0
    date_created:      2018-12-18
    creator_name:      Pete Peterson
    ...                ...
    reference:         Funk, C.C., Peterson, P.J., Landsfeld, M.F., Pedreros,...
    comments:           time variable denotes the first day of the given month.
    acknowledgements:  The Climate Hazards Group InfraRed Precipitation with ...
    ftp_url:           ftp://chg-ftpout.geog.ucsb.edu/pub/org/chg/products/CH...
    website:           http://chg.geog.ucsb.edu/data/chirps/index.html
    faq:               http://chg-wiki.geog.ucsb.edu/wiki/CHIRPS_FAQ

### General metadata

In [3]:
metadata ={}
metadata['title'] = data.attrs['title']
metadata['description'] = 'Climate Hazards Group InfraRed Precipitation with Station data (CHIRPS) is a 35+ year quasi-global rainfall data set. Spanning 50°S-50°N (and all longitudes) and ranging from 1981 to near-present, CHIRPS incorporates our in-house climatology, CHPclim, 0.05° resolution satellite imagery, and in-situ station data to create gridded rainfall time series for trend analysis and seasonal drought monitoring.'
metadata['source_url'] = data.attrs['website']
metadata['version'] = data.attrs['version']
metadata['file_url'] = 'https://data.mint.isi.edu/CHIRPS/2.0/global_monthly/chirps-v2.0.monthly.nc'

### Geographical coverage

In [8]:
geo={}
geo['type'] = 'BoundingBox'
geo['value'] = {}
geo['value']['xmin'] = float(data.longitude.min())
geo['value']['xmax'] = float(data.longitude.max())
geo['value']['ymin'] = float(data.latitude.min())
geo['value']['ymax'] = float(data.latitude.max())
geo['resolution'] = "{0:.2f}".format(np.diff(data.longitude.values).mean()) + 'deg x' + "{0:.2f}".format(np.diff(data.latitude.values).mean()) +'deg'

metadata['spatial_coverage'] = geo

### Temporal coverage

In [5]:
t = {}
t['start_time'] = np.datetime_as_string(data.time.min().values)
t['end_time'] = np.datetime_as_string(data.time.max().values)
t['resolution'] = data.precip.attrs['time_step']

metadata['temporal_coverage'] = t

### Variables

In [6]:
varname = list(data.data_vars.keys())
variables={}

for item in varname:
    s={}
    s['long_name']=data[item].attrs['long_name']
    s['standard_name']=data[item].attrs['long_name']
    variables[item] = s

metadata['variables'] = variables

### Check your work

In [9]:
metadata

{'title': 'CHIRPS Version 2.0',
 'description': 'Climate Hazards Group InfraRed Precipitation with Station data (CHIRPS) is a 35+ year quasi-global rainfall data set. Spanning 50°S-50°N (and all longitudes) and ranging from 1981 to near-present, CHIRPS incorporates our in-house climatology, CHPclim, 0.05° resolution satellite imagery, and in-situ station data to create gridded rainfall time series for trend analysis and seasonal drought monitoring.',
 'source_url': 'http://chg.geog.ucsb.edu/data/chirps/index.html',
 'version': 'Version 2.0',
 'file_url': 'https://data.mint.isi.edu/CHIRPS/2.0/global_monthly/chirps-v2.0.monthly.nc',
 'spatial_coverage': {'type': 'BoundingBox',
  'value': {'xmin': -179.97500610351562,
   'xmax': 179.97500610351562,
   'ymin': -49.974998474121094,
   'ymax': 49.974998474121094},
  'resolution': '0.05deg x0.05deg'},
 'temporal_coverage': {'start_time': '1981-01-01T00:00:00.000000000',
  'end_time': '2018-11-01T00:00:00.000000000',
  'resolution': 'month'},


## Export to JSON

In [10]:
with open("chirps_monthly.json", "w") as outfile: 
    json.dump(metadata, outfile)